# Copernicus Atmosphere Monitoring Service (CAMS) data examples

The Copernicus Atmopshere Monitoring Service (CAMS) provides consistent and quality-controlled information related to air pollution and health and greenhouse gases. CAMS data consist of `global forecasts and analyses`, `global reanalyses`, `fire emissions` and `greenhouse gas flux inversions`.

In [ ]:
%matplotlib inline
import os
import xarray as xr
import numpy as np
import netCDF4 as nc
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.colors
from matplotlib.cm import get_cmap
import cartopy
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.feature as cfeature

from matplotlib.axes import Axes
from cartopy.mpl.geoaxes import GeoAxes
GeoAxes._pcolormesh_patched = Axes.pcolormesh

import warnings
warnings.simplefilter(action = "ignore", category = RuntimeWarning)

<hr>

## <a id="cams_gfas"></a>Load, browse and plot CAMS fire emissions data

### Open a CAMS GFAS netCDF file with `xarray`

CAMS GFAS fire emission data can be retrieved in either `GRIB` or `NetCDF` format. With the Python library `xarray` and the `open_dataset` function, we can easily read a single `NetCDF` file.

CAMS GFAS fire emission data are three dimensional data, with the dimensions `latitude`, `longitude` and `time`. The data file loaded has three time steps, from 22 September to 24 September 2019 and a global spatial coverage. The xarray dataset contains a data variable called `frpfire`.

In [ ]:
gfas_frpfire_xr = xr.open_dataset('../eodata/cams/gfas_fire_radiative_power_20191110.nc')
gfas_frpfire_xr

We can select the data variable with `[]`, which gives us acces to the DataArray and more parameter attributes. Thus, the dataset values are `wildfire radiative power` and the parameter unit is `W m**-2`.

In [ ]:
frpfire = gfas_frpfire_xr['frpfire']
frpfire

With xarray's `where` function, we can build a geographical subset. Let's define a bounding box for south-east asia `[30,160,-20,80]` and subset the data. We see that the data has a negative _FillValue. Thus a next step is to filter negative values and set them to NaN.

In [ ]:
latmin=-50
latmax=-5
lonmin=110
lonmax=160
frpfire_subset = frpfire.where((frpfire.latitude < latmax) & (frpfire.latitude > latmin) & (frpfire.longitude < lonmax) & (frpfire.longitude > lonmin),drop=True)
frpfire_subset

In [ ]:
frpfire_subset.values[frpfire_subset.values<0]=np.nan
frpfire_subset

### Visualize CAMS GFAS fire emissions data

You can use the function [visualize_pcolormesh](./ltpy_functions.ipynb#visualize_pcolormesh) again. This time, you should set a logarithmic color scaling with setting `log_scale=True`. 
With `matplotlib.colors.ListedColorMap`, you can specify an individual color scale. Let's call the color scale `cmap` and plot the fire emissions data. Alternatively, you can use the predefine color scale `hot`.

In [ ]:
unit = frpfire.units
longname= frpfire.long_name

In [ ]:
import cartopy
import cartopy.crs as ccrs
from cartopy.feature import LAND
from matplotlib import colors, cm

map_proj = ccrs.PlateCarree()
BORDER_COLOUR = "#cccccc"

f, ax = plt.subplots(figsize=(16, 7), subplot_kw=dict(projection=map_proj))

ax.add_feature(LAND)
ax.coastlines()

p = frpfire_subset[0,:,:].plot(ax=ax, transform=ccrs.PlateCarree(), vmin=0.00001, vmax=1, cmap=plt.get_cmap('hot', 10),
  norm=colors.LogNorm(vmin=0.00001, vmax=1), extend="max")
ax.add_feature(cartopy.feature.BORDERS, color=BORDER_COLOUR)
ax.coastlines(color=BORDER_COLOUR)
# ax.set_extent((105, 160, -50, 0))
ax.gridlines(draw_labels=True, crs=ccrs.PlateCarree() )
plt.title("CAMS GFAS fire radiative power 2019-11-10")
plt.show()

<br>

<br>